In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
user= pd.read_csv("users.csv")

In [3]:
movies=pd.read_csv("Movies.csv")

In [4]:
user

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742
...,...,...,...,...
99998,880,476,3,880175444
99999,716,204,5,879795543
100000,276,1090,1,874795795
100001,13,225,2,882399156


In [5]:
user.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [7]:
movies

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [9]:
df= pd.merge(user, movies)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100003 entries, 0 to 100002
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    100003 non-null  int64 
 1   item_id    100003 non-null  int64 
 2   rating     100003 non-null  int64 
 3   timestamp  100003 non-null  int64 
 4   title      100003 non-null  object
dtypes: int64(4), object(1)
memory usage: 3.8+ MB


In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,0,172,5,881250949,"Empire Strikes Back, The (1980)"
2,0,133,1,881250949,Gone with the Wind (1939)
3,196,242,3,881250949,Kolya (1996)
4,186,302,3,891717742,L.A. Confidential (1997)


In [13]:
df.drop("timestamp", axis=1, inplace=True)

In [16]:
df.groupby("title")["rating"].mean().sort_values(ascending=False)

title
Aiqing wansui (1994)                                 5.0
Entertaining Angels: The Dorothy Day Story (1996)    5.0
Santa with Muscles (1996)                            5.0
Prefontaine (1997)                                   5.0
They Made Me a Criminal (1939)                       5.0
                                                    ... 
Pharaoh's Army (1995)                                1.0
August (1996)                                        1.0
Babyfever (1994)                                     1.0
Amityville: A New Generation (1993)                  1.0
Woman in Question, The (1950)                        1.0
Name: rating, Length: 1664, dtype: float64

In [19]:
df["title"].value_counts()<100

title
Star Wars (1977)                              False
Contact (1997)                                False
Fargo (1996)                                  False
Return of the Jedi (1983)                     False
Liar Liar (1997)                              False
                                              ...  
Eighth Day, The (1996)                         True
Mamma Roma (1962)                              True
Scream of Stone (Schrei aus Stein) (1991)      True
Getting Away With Murder (1996)                True
Nobody Loves Me (Keiner liebt mich) (1994)     True
Name: count, Length: 1664, dtype: bool

In [20]:
movie_counts= df["title"].value_counts().reset_index()

In [23]:
top_rated_movies= movie_counts.query("count >= 100")["title"].tolist()

In [26]:
dummy= df[df["title"].isin(top_rated_movies)]

In [28]:
dummy["title"].value_counts()

title
Star Wars (1977)                          584
Contact (1997)                            509
Fargo (1996)                              508
Return of the Jedi (1983)                 507
Liar Liar (1997)                          485
                                         ... 
Young Guns (1988)                         101
Alien 3 (1992)                            100
Muriel's Wedding (1994)                   100
I Know What You Did Last Summer (1997)    100
Private Parts (1997)                      100
Name: count, Length: 338, dtype: int64

In [29]:
dummy.groupby('title')["rating"].mean().sort_values(ascending=False)

title
Close Shave, A (1995)               4.491071
Schindler's List (1993)             4.466443
Wrong Trousers, The (1993)          4.466102
Casablanca (1942)                   4.456790
Shawshank Redemption, The (1994)    4.445230
                                      ...   
Spawn (1997)                        2.615385
Event Horizon (1997)                2.574803
Crash (1996)                        2.546875
Jungle2Jungle (1997)                2.439394
Cable Guy, The (1996)               2.339623
Name: rating, Length: 338, dtype: float64

In [31]:
user_matrix= pd.pivot_table(dummy, index="user_id", columns="title", values="rating", fill_value=0)

In [33]:
movies_corr= user_matrix.corr()

In [35]:
movies_corr["Schindler's List (1993)"].sort_values(ascending=False).head(6)[1:]

title
Forrest Gump (1994)                  0.522185
Dances with Wolves (1990)            0.508768
Shawshank Redemption, The (1994)     0.506069
When Harry Met Sally... (1989)       0.473037
E.T. the Extra-Terrestrial (1982)    0.471581
Name: Schindler's List (1993), dtype: float64

In [36]:
def movie_recommender(movie):
    return movies_corr[movie].sort_values(ascending=False).head(6)[1:]

In [37]:
movie_recommender("Event Horizon (1997)")

title
Mimic (1997)                  0.365647
Alien: Resurrection (1997)    0.290812
Edge, The (1997)              0.282791
Volcano (1997)                0.270164
Spawn (1997)                  0.262289
Name: Event Horizon (1997), dtype: float64

In [38]:
movie_recommender("Event Horizon (1997)").index

Index(['Mimic (1997)', 'Alien: Resurrection (1997)', 'Edge, The (1997)',
       'Volcano (1997)', 'Spawn (1997)'],
      dtype='object', name='title')

In [46]:
def user_id_list(movie_name):
    return set(dummy[(dummy["title"].isin(movie_recommender(movie_name).index)) & 
                  (~dummy["title"].isin([movie_name]))]["user_id"].tolist())

In [47]:
len(user_id_list("Event Horizon (1997)"))

399

In [44]:
movie_matrix= pd.pivot_table(dummy, index="title", columns="user_id", values="rating", fill_value=0)

In [45]:
movie_matrix

user_id,0,1,2,3,4,5,6,7,8,9,...,934,935,936,937,938,939,940,941,942,943
title,,,,,,,,,,,,,,,,,,,,,
101 Dalmatians (1996),0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,5.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,4.0,0.0,0.0,0.0,4.0,5.0,5.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
Absolute Power (1997),0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
"Abyss, The (1989)",0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Willy Wonka and the Chocolate Factory (1971),0.0,4.0,0.0,0.0,0.0,3.0,3.0,4.0,0.0,0.0,...,3.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,4.0
"Wizard of Oz, The (1939)",0.0,4.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
"Wrong Trousers, The (1993)",0.0,5.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
user_corr= movie_matrix.corr()

In [49]:
user_corr

user_id,0,1,2,3,4,5,6,7,8,9,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.119795,0.111087,-0.024751,0.185292,0.167661,0.027793,0.075264,0.223369,0.203732,...,0.152852,-0.026088,0.051701,0.163922,0.080232,-0.025099,0.138994,-0.019970,0.190657,0.107554
1,0.119795,1.000000,-0.054261,-0.174900,-0.077941,0.243078,0.223382,0.174818,0.214558,-0.096754,...,0.245160,-0.068712,-0.002040,0.042228,-0.103845,-0.068293,0.095377,0.050027,-0.058466,0.226611
2,0.111087,-0.054261,1.000000,0.041123,0.192187,-0.079160,0.112407,-0.176112,-0.005781,0.177655,...,0.024462,0.320928,0.403441,0.448704,0.297926,0.221100,0.133281,0.133072,0.098932,-0.068617
3,-0.024751,-0.174900,0.041123,1.000000,0.385212,-0.124788,-0.089820,-0.193639,-0.023045,0.042234,...,-0.126703,-0.026391,0.184488,0.026195,0.058330,-0.043854,0.060679,0.078801,0.088167,-0.148253
4,0.185292,-0.077941,0.192187,0.385212,1.000000,-0.068370,-0.048101,-0.045944,0.175958,0.117834,...,-0.044107,-0.016168,0.145758,0.231297,0.119851,-0.018673,0.130808,0.160397,0.139429,-0.068390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,-0.025099,-0.068293,0.221100,-0.043854,-0.018673,-0.052703,-0.019697,-0.100944,-0.012922,0.003084,...,-0.087694,0.486941,0.285651,0.264024,0.439152,1.000000,-0.032396,0.146490,-0.084828,0.033381
940,0.138994,0.095377,0.133281,0.060679,0.130808,0.046024,0.192481,0.082902,0.114544,0.073328,...,0.140577,-0.024392,0.065172,0.109270,-0.025193,-0.032396,1.000000,0.064923,0.149723,0.026546
941,-0.019970,0.050027,0.133072,0.078801,0.160397,0.089488,0.059181,-0.141900,0.087856,0.156667,...,-0.076719,0.187983,0.292641,0.199637,0.283520,0.146490,0.064923,1.000000,0.035673,-0.012814


In [58]:
def similar_user(user_id):
    return user_corr[user_id].sort_values(ascending=False).head(6)[1:].index

In [59]:
similar_user(123)

Index([6, 383, 321, 492, 382], dtype='int64', name='user_id')

In [62]:
def movie_list(user_id):
    return dummy[(dummy["user_id"].isin(similar_user(user_id))) & (dummy["user_id"]!=123)]["title"].value_counts().head(5).index.tolist()

In [63]:
movie_list(123)

['Citizen Kane (1941)',
 'English Patient, The (1996)',
 'Fargo (1996)',
 'Annie Hall (1977)',
 'Shine (1996)']